We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.

In [7]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import (
    Grad
)  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import Elastic_Functions as elfunc
import General_Functions as genfunc
import numpy as np
import scipy.sparse as sp
import time
T_MAX: float = 10  # The maximum time for the simulation
ALPHA: float = 5  # Dissipative constant in the LLG equation.
THETA: float = 0.505  # Should be strictly above 1/2 for unconditional stability
K: float = 0.1  # TIME STEP
KAPPA: float = 1.0  # Determines the relative strength of the elastic vs. magnetic parts.
mu: float = 1  # first lame constant
lam: float = 1  # second lame constant
lambda_m: float = 1  # saturation magnetostrain parameter
H_MAX: float = 0.5  # Determines how fine the mesh should be.

In [8]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left = Plane(Pnt(0, 0, 0), Vec(-1, 0, 0)).bc("left")
    right = Plane(Pnt(1, 1, 1), Vec(1, 0, 0)).bc("right")
    front = Plane(Pnt(0, 0, 0), Vec(0, -1, 0)).bc("front")
    back = Plane(Pnt(1, 1, 1), Vec(0, 1, 0)).bc("back")
    bot = Plane(Pnt(0, 0, 0), Vec(0, 0, -1)).bc("bot")
    top = Plane(Pnt(1, 1, 1), Vec(0, 0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add(cube)
    # cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry


ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [9]:
fes_mag = VectorH1(
    mesh, order=1
)  # the finite element space for the magnetisation m_h^i
fes_eps_m = MatrixValued(
    H1(mesh, order=1), dim=3
)  # matrix FE space on the magnetic part
fes_disp = VectorH1(
    mesh, order=1, dirichlet="bot"
)  # the finite element space for the displacement u_h^i

# print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
prev_disp_gfu = GridFunction(fes_disp) #  used to store the previous displacement
# body force and traction force
body_factor = Parameter(-1.0)
f_body = CoefficientFunction( (0, 0, body_factor) )
surface_factor = Parameter(1)
g_surface = CoefficientFunction([(0,0,surface_factor) if bc=="left" else 0 for bc in mesh.GetBoundaries()])

In [10]:
# Initial conditions
mag_gfu = magfunc.give_random_magnetisation(mag_gfu)
disp_gfu = elfunc.give_random_displacement(disp_gfu)
velocity_gfu = GridFunction(fes_disp)  # velocity
velocity_gfu = elfunc.give_random_displacement(velocity_gfu)  # An initial velocity. Should only be used once in iteration.
Draw(mag_gfu)
magfunc.magnetic_energy(mag_gfu, mesh)

14.37856619084823

In [11]:
genfunc.export_to_vtk_file(disp_gfu, mag_gfu, mesh, export=False, index=0)

In [12]:
num_steps = genfunc.ceiling_division(T_MAX, K)  # This is ceiling division, using upside-down floor division.
#first run
proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)
strain_m = magfunc.build_strain_m(proj_mag, lambda_m)

mag_gfu = magfunc.update_magnetisation(fes_mag, mag_gfu, fes_eps_m, ALPHA, THETA, K, KAPPA, disp_gfu, mu, lam, lambda_m)

proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)
strain_m = magfunc.build_strain_m(mag_gfu, lambda_m)
prev_disp_gfu.vec.data = disp_gfu.vec.data
disp_gfu = elfunc.FIRST_RUN_update_displacement(fes_disp, disp_gfu, velocity_gfu, strain_m, f_body, g_surface, K, mu, lam)
# subsequence runs
for i in range(1,num_steps):
    proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)
    strain_m = magfunc.build_strain_m(proj_mag, lambda_m)
    mag_gfu = magfunc.update_magnetisation(fes_mag, mag_gfu, fes_eps_m, ALPHA, THETA, K, KAPPA, disp_gfu, mu, lam, lambda_m)
    proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)
    strain_m = magfunc.build_strain_m(proj_mag, lambda_m)
    new_disp = elfunc.update_displacement(fes_disp, disp_gfu, prev_disp_gfu, strain_m, f_body, g_surface, K, mu, lam)
    prev_disp_gfu.vec.data = disp_gfu.vec.data
    disp_gfu.vec.data = new_disp.vec.data
    Redraw()
    print(f"Step {i}:integral |m*m|dx = {Integrate(mag_gfu*mag_gfu, mesh, VOL)} (should be 1)\n\
        energy      = {magfunc.magnetic_energy(mag_gfu, mesh) + elfunc.elastic_energy(mesh, disp_gfu, strain_m, f_body, g_surface, KAPPA, mu, lam)}")
    vtk = VTKOutput(
        ma=mesh,
        coefs = [mag_gfu],
        names = ["magnetisation"],
        filename = f"the_result{i+1}"
    )
    vtk.Do()

gmres completed in 0.003020763397216797, info=0, residual = 5.59082175000114e-09
gmres completed in 0.002961874008178711, info=0, residual = 4.0412303031756025e-08
Step 1:integral |m*m|dx = 2.189794906974606 (should be 1)
        energy      = 2395.6287428733854
gmres completed in 0.016953229904174805, info=0, residual = 8.195771350472114e-08
Step 2:integral |m*m|dx = 9.80867026438024 (should be 1)
        energy      = 3355.890809430763
gmres completed in 0.0528867244720459, info=0, residual = 1.1852049297544909e-07
Step 3:integral |m*m|dx = 35.74252435412303 (should be 1)
        energy      = 4661.846400263568
gmres completed in 0.1216433048248291, info=0, residual = 1.21802724795117e-07
Step 4:integral |m*m|dx = 93.38861922335022 (should be 1)
        energy      = 6635.32219104084
gmres completed in 0.23234868049621582, info=0, residual = 9.953119922689879e-08
Step 5:integral |m*m|dx = 145.74964396979445 (should be 1)
        energy      = 9162.513863463266
gmres completed in 0.15